## Imports

In [ ]:
import mlflow
from mlflow import MlflowClient
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature
from transformers import pipeline
import torch
import json
import os

## Model

In this part of the code, we load a Transformer model saved on Hugging Face to use it locally (in a pipeline object). This pipeline is then tested with a simple sample.

In [ ]:
model_name = "morgana-rodrigues/bert_qa"

qa_pipeline = pipeline(
    'question-answering',
    model=model_name,
    device=0 # -1 means running on CPU
)

In [ ]:
qa_pipeline (context="Take me down to Paradise City where the grass is green and the girls are pretty", question="What colour is the grass")

This class below encapsulates the model in the format that will be logged/registered into MLFlow. It receives a pipeline (or a trainer) as input, saves the model into a temporary folder (called model_name), and log as an artifact into MLFlow. When MLFlow deploys the model, it loads these artifacts into a new pipeline, which can be used to perform inference.

In [ ]:
class DistilBERTModel(mlflow.pyfunc.PythonModel):
    def _preprocess(self, inputs):
        context = inputs['context'][0]
        question = inputs['question'][0]
        print("pre processing", context,question)
        return context, question
        
    def load_context(self, context):
        self.model = pipeline(
            'question-answering',
             model=context.artifacts["model"],
             device=0
        )
        
    def predict(self, context, model_input, params):
        in_ctx, question = self._preprocess(model_input)
        output = self.model(context=in_ctx, question=question)
        return output

    @classmethod
    def log_model(cls, model_name, source_trainer = None, source_pipeline = None, demo_folder="demo"): 
        import shutil
        input_schema = Schema(
            [
                ColSpec("string", "context"),
                ColSpec("string", "question"),
            ]
        )
        output_schema = Schema(
            [
                ColSpec("string", "answer")
            ]
        )
        
        params_schema = ParamSchema(
            [
                ParamSpec("show_score", "boolean", False)
            ]
        )
      
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        if source_trainer is not None:
            source_trainer.save_model(model_name)
        elif source_pipeline is not None:
            source_pipeline.save_pretrained(model_name)
             
        requirements = [
            "transformers==4.47.0",
            "tf_keras"
        ]
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={"model": model_name, "demo": demo_folder},
            signature=signature,
            pip_requirements=requirements
        )
        shutil.rmtree(model_name)

## Model Registry

In [ ]:
mlflow.set_experiment(experiment_name='BERT model for Q&A')

In [ ]:
with mlflow.start_run(run_name='BERT_QA') as run:
    print(f"Run's Artifact URI: {run.info.artifact_uri}")
    DistilBERTModel.log_model(model_name='BERT_QA', source_pipeline=qa_pipeline)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/BERT_QA", name='BERT_QA')

## Testing latest model registred

In [ ]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions("BERT_QA", stages=["None"])
latest_model_version = model_metadata[0].version
print(latest_model_version, mlflow.models.get_model_info(f"models:/BERT_QA/{latest_model_version}").signature)

In [ ]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/BERT_QA/{latest_model_version}")
context = "Marta is mother of John and Amanda"
question = "what is the name of Marta's daugther?"
model.predict({"context": [context], "question":[question]})